In [240]:
import pandas as pd
import sys
sys.path
sys.path.append('../models')
sys.path.append('../simulation')
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study
import time as time_module
from scipy.linalg import block_diag
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
from numpy.random import uniform

#sys.path.append('../simulation')
import TS_fancy_pooled 
import TS
#import TS_fancy_pooled 
import eta
import pooling_bandits as pb
import simple_bandits
import warnings 
warnings.simplefilter('ignore')
from sklearn import preprocessing
#import tensorflow as tf
import gc
import standard_bandits as do_work
import feature_transformations as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [90]:
with open('../simulation/test_this_history.pkl','rb') as f:
    rhist = pickle.load(f)

In [91]:
def initialize_policy_params_TS(experiment,update_period,standardize=False):
    
    global_p =gtp.TS_global_params(21,baseline_keys=['tod','dow','pretreatment','location'],psi_features=[0,5], responsivity_keys= ['tod','dow','pretreatment','location'])
    personal_p = pp.TS_personal_params()
  
    
    global_p.standardize = standardize
    global_p.kdim =21

    #global_p.baseline_indices = [i for i in range(24)]

    global_p.psi_indices =[0,5]
    #[0,64]
    global_p.user_id_index =24
    #192
    #global_p.user_day_index =19
    #193
    
    #global_p.baseline_features = [i for i in range(192)]
    global_p.psi_features =[0,5]
    #[0,64]
    
    global_p.update_period = update_period
    initial_context = [0 for i in range(global_p.theta_dim)]
    
    global_p.mus0= global_p.get_mu0(initial_context)
    #global_p.get_mu0(initial_context)
    global_p.mus1= global_p.get_mu1(global_p.num_baseline_features)
    global_p.mus2= global_p.get_mu2(global_p.num_responsivity_features)
    #np.array([.120,3.3,-.11])
    #global_p.get_mu2(global_p.num_responsivity_features)
        
    #global_p.sigmas0= global_p.get_asigma(len( personal_p.mus0[person]))
    global_p.sigmas1= global_p.get_asigma(global_p.num_baseline_features+1)
    global_p.sigmas2= global_p.get_asigma( global_p.num_responsivity_features+1)
        
        
    
    #print(type(personal_p))
    
    for person in experiment.population.keys():
        experiment.population[person].root = '../../regal/murphy_lab/pooling/distributions/'
        

        
        personal_p.batch[person]=[[] for i in range(len(experiment.person_to_time[person]))]
        personal_p.batch_index[person]=0
        
        #personal_p.etas[person]=eta.eta()
        
        personal_p.last_update[person]=experiment.person_to_time[person][0]
        
        
    return global_p ,personal_p     

In [92]:
##check one the history is similar
time = pd.Timestamp('2015-08-17 19:00:00')

glob,personal = initialize_policy_params_TS(rhist,7,standardize=False)
feat_trans = tf.feature_transformation('../../Downloads/distributions/')


15


In [93]:
##baseline

temp_hist = feat_trans.get_history_decision_time_avail(rhist,time)
temp_hist= feat_trans.history_semi_continuous(temp_hist,glob)
#no intercept but that is added in the other function so it is used in policy update

context,steps,probs,actions= feat_trans.get_form_TS(temp_hist)
                
temp_data = feat_trans.get_phi_from_history_lookups(temp_hist)
                #print(context)
               # print(steps)
steps = feat_trans.get_RT_o(steps,temp_data[0],glob.mu_theta,glob.theta_dim)
print(steps.mean())
print(steps.std())
                #print(len(context))
                #print(len(steps))
                
                ##why is it sigma here and not the noise, which should it be?


5.124978588962189
1.1885715755800625


In [ ]:
steps = feat_trans.get_RT_o(steps,temp_data[0],glob.mu_theta,glob.theta_dim)

In [ ]:
inv_Sigma = solve(Sigma,np.eye(len(Sigma[0])))

In [336]:
posterior_baseline= TS.policy_update_ts_new( context,steps,probs,actions,glob.noise_term**.5,\
                                               glob.mus1,\
                                               glob.sigmas1,\
                                               glob.mus2,\
                                               glob.sigmas2,   
                                              )


In [185]:
        f_one = TS.transform_f1_new(context)
        f_two = TS.transform_f2_new(context)
        
        X_trn = TS.get_X_trn_new(f_one,actions,f_two,probs)

In [ ]:
 0.08360331, -0.05316288,  0.30300525,  0.07435143,  0.11629954

In [337]:
posterior_baseline

[array([ 0.08360331, -0.05316288,  0.30300525,  0.07435143,  0.11629954]),
 array([[ 0.09745588, -0.02546992, -0.01280663, -0.07030492, -0.02253609],
        [-0.02546992,  0.05045578, -0.0012546 , -0.00505827,  0.00043247],
        [-0.01280663, -0.0012546 ,  0.06964607,  0.002033  , -0.0114217 ],
        [-0.07030492, -0.00505827,  0.002033  ,  0.08492448,  0.00598308],
        [-0.02253609,  0.00043247, -0.0114217 ,  0.00598308,  0.05457523]])]

In [241]:
Sigma_tmp = block_diag(glob.sigmas1,glob.sigmas2,glob.sigmas2)

In [246]:
inv_Sigma = np.linalg.solve(Sigma_tmp,np.eye(len(Sigma_tmp[0])))

In [304]:
    term_one = np.dot(np.transpose(history[0]),history[0])+(glob.noise_term**.5)*inv_Sigma
    term_two = np.dot(np.transpose(history[0]),history[2])+np.dot((glob.noise_term**.5)*inv_Sigma,np.eye(15))
    pos_mean = np.linalg.solve(term_one,term_two)

In [314]:
term_two.shape

(15, 15)

In [96]:
#pooled
temp =feat_trans.get_history_decision_time_avail(rhist,time)
                    #print(history[1])
t = feat_trans.history_semi_continuous(temp,glob)
history = feat_trans.get_phi_from_history_lookups(t)
y_adjusted = feat_trans.get_RT_o(history[2],history[0],glob.mu_theta,glob.theta_dim)


In [200]:
history[2].std()

1.1885715755800625

In [315]:
def get_cov(history,sigma_theta):
    return np.dot(np.dot(history,sigma_theta),history.T)

In [316]:
cov = get_cov(history[0],glob.sigma_theta)

In [354]:
cov

array([[3.  , 3.  , 3.  , ..., 2.  , 3.  , 3.  ],
       [3.  , 3.  , 3.  , ..., 2.  , 3.  , 3.  ],
       [3.  , 3.  , 4.5 , ..., 2.  , 4.5 , 4.5 ],
       ...,
       [2.  , 2.  , 2.  , ..., 6.84, 4.44, 4.44],
       [3.  , 3.  , 4.5 , ..., 4.44, 6.72, 6.72],
       [3.  , 3.  , 4.5 , ..., 4.44, 6.72, 6.72]])

In [321]:
def get_inv_term(cov,X_dim,noise_term):
    noise = noise_term * np.eye(cov.shape[0])
    middle_term = np.add(cov,noise)
    #inv_term = np.linalg.inv(middle_term)
    return np.linalg.inv(middle_term)

In [323]:
inv_term = get_inv_term(cov,15,glob.noise_term)
glob.inv_term = inv_term
#np.linalg.inv(glob.noise_term*np.eye(history[0].shape[0]))

In [338]:
###I want to compare this step to the policy update, if I set everything to zero will it                               
temp = simple_bandits.calculate_posterior_faster(glob,\
                                                  0,0,\
                                                  history[0], history[1],y_adjusted )




In [326]:




def get_M_faster(global_params,user_id,user_study_day,history,users):
    
    
    day_id =user_study_day
    #print(history)
    M = [[] for i in range(history.shape[0])]
    
    H = simple_bandits.create_H(global_params.num_baseline_features,global_params.num_responsivity_features)
    
    phi = history[:,global_params.baseline_indices]
    ##should be fine
    #print(global_params.sigma_theta)
    t_one = np.dot(phi,global_params.sigma_theta)
    #print(t_one.shape)
    temp = np.dot(H,global_params.sigma_u)
    #print(temp.shape)
    #print(global_params.sigma_u)
    temp = np.dot(temp,H.T)
    temp = np.dot(phi,temp)
    
    user_ids =users
    #history[:,global_params.user_id_index]

    my_days = np.ma.masked_where(user_ids==user_id, user_ids).mask.astype(float)
    
    if type(my_days)!=np.ndarray:
        my_days = np.zeros(history.shape[0])
    user_matrix = np.diag(my_days)

    t_two = np.matmul(user_matrix,temp)
    #print(set([i for i in t_two[1000]]))
    term = np.add(t_one,t_two)
    
    
    return term


In [330]:
def get_middle_term(X_dim,cov,noise_term,M,adjusted_rewards,mu_theta,inv_term):
  
    middle_term = np.matmul(M.T,inv_term)
    print(middle_term.shape)
    #return middle_term
    #return middle_term
    #print(middle_term)
    print(inv_term.shape)
    print(adjusted_rewards.shape)
    middle_term = np.matmul(middle_term,adjusted_rewards)
    #print(middle_term)
    print(middle_term.shape)
    #return middle_term
    return np.add(mu_theta,middle_term.T[0])

In [353]:
H = simple_bandits.create_H(glob.num_baseline_features,glob.num_responsivity_features)
M = get_M_faster(glob,2,-1,history[0],history[1])
mu = get_middle_term(2,-1,10,M,np.array([i for i in history[2]]),glob.mu_theta,glob.inv_term)
mu[-5:]

(15, 2783)
(2783, 2783)
(2783, 1)
(15, 1)


array([ 0.08360331, -0.05316288,  0.30300525,  0.07435143,  0.11629954])

In [376]:
def get_post_sigma(H,cov,sigma_u,sigma_v,noise_term,M,x_dim,sigma_theta,inv_term):
    #M = get_M(global_params,user_id,user_study_day,history[0])
    
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    #adjusted_rewards =[history[1][i]-np.dot(history[0][i][0:6],np.ones(6)) for i in range(len(history[0]))]
    
    
    
    #first_term = np.add(sigma_u,sigma_v)
    first_term = sigma_u
   
    first_term = np.dot(H,first_term)

    first_term = np.dot(first_term,H.T)
    #print(first_term)
    
    #noise = noise_term * np.eye(x_dim)
    #print(noise.shape)
    #middle_term = np.add(cov,noise)
    #print(middle_term.shape)
    middle_term = np.dot(M.T,inv_term)
    #print(middle_term.shape)
    middle_term = np.dot(middle_term,M)
    #print(middle_term.shape)
    last = np.add(sigma_theta,first_term)
    last = np.subtract(last,middle_term)
    
    return last

In [377]:
sigma = get_post_sigma(H,cov,glob.sigma_u.reshape(2,2),None,glob.noise_term,M,history[0].shape[0],glob.sigma_theta,glob.inv_term)

In [378]:
stuff = []
for row in range(10,15):
    temp = []
    for column in range(10,15):
        temp.append(sigma[row][column])
    stuff.append(temp)

In [379]:
stuff

[[0.09745587947806533,
  -0.025469924653243013,
  -0.012806625753481037,
  -0.07030492324433216,
  -0.022536094346316617],
 [-0.02546992465324508,
  0.05045577614007846,
  -0.0012546047838403807,
  -0.005058265196319495,
  0.0004324715363728077],
 [-0.012806625753481687,
  -0.0012546047838398562,
  0.06964606619671421,
  0.0020329995729350472,
  -0.011421697858617592],
 [-0.07030492324433063,
  -0.005058265196317997,
  0.0020329995729349536,
  0.08492447755579302,
  0.005983080967406025],
 [-0.0225360943463175,
  0.00043247153637395765,
  -0.011421697858617135,
  0.005983080967405893,
  0.05457523218297167]]

In [ ]:
 array([[ 0.09745588, -0.02546992, -0.01280663, -0.07030492, -0.02253609],
        [-0.02546992,  0.05045578, -0.0012546 , -0.00505827,  0.00043247],
        [-0.01280663, -0.0012546 ,  0.06964607,  0.002033  , -0.0114217 ],
        [-0.07030492, -0.00505827,  0.002033  ,  0.08492448,  0.00598308],
        [-0.02253609,  0.00043247, -0.0114217 ,  0.00598308,  0.05457523]])]

In [352]:
glob.sigma_u = np.array([[0,0],[0,0]])

In [308]:
mt = np.dot(M.T,glob.inv_term)
t2 = np.dot(mt,history[2])
#np.linalg.solve(np.dot(M.T,glob.inv_term),)

In [312]:
#np.linalg.solve(history[2],mt)

LinAlgError: Last 2 dimensions of the array must be square

In [206]:
# [history[2][i]-np.dot(history[0][i],glob.mu_theta) for i in range(len(history[0]))]

In [207]:
#np.dot(history[0][0].T,glob.sigma_theta)

array([1. , 0. , 0. , 1. , 0. , 0.5, 0. , 0. , 0.5, 0. , 0.5, 0. , 0. ,
       0.5, 0. ])

In [208]:
mu

array([1873.49431207, 1095.41811558,  513.15342009, 1642.86756791,
        766.69714323, 1233.2406518 ,  715.23285641,  373.10480605,
       1094.46307386,  529.54732496,   14.29586599,    6.19407692,
         13.05362984,   16.41992942,   11.13901505])